In [42]:
#load the required modules
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from numba import njit, prange
import warnings
import plotly.graph_objects as go
import plotly.subplots as sp

In [43]:
np.random.seed(43)

In [44]:
@njit(fastmath=True)
def vec_mag(vec):
    """
    This function calculates the magnitude of a 3D vector.

    Parameters:
    vec (np.ndarray): A 3D vector represented as a numpy array.

    Returns:
    float: The magnitude of the vector.
    """
    return np.sqrt(np.sum(vec**2))

In [45]:
@njit(fastmath=True)
def proton_points(R_val):
    """
    This function generates the positions of two protons in 3D space by
    assuming they are aligned along the z-axis.

    Parameters:
    R_val: distance between the protons

    Returns:
    P1, P2: Two points in 3D space representing the positions of the protons.
    """
    P1 = np.array([0, 0, -R_val/2.0])
    P2 = np.array([0, 0, R_val/2.0])
    return P1, P2

In [46]:
@njit(fastmath=True)  
def single_electron_wavefunction(r_e, R_val, alpha, P1, P2, sign=1):
    """
    This function calculates the wavefunction of a
    single electron in the field of two protons.

    Parameters:
    r_e (np.ndarray): The position of the electron in 3D space.
    R_val (float): The distance between the two protons.
    alpha (float): The variational parameter.
    P1 (np.ndarray): The position of the first proton in 3D space.
    P2 (np.ndarray): The position of the second proton in 3D space.
    sign (int): The sign of the wavefunction.

    Returns:
    float: The value of the wavefunction at the given electron position.
    """
    r1 = vec_mag(r_e - P1)
    r2 = vec_mag(r_e - P2)
    #avoid division by zero
    if r1 < 1e-9: r1 = 1e-9
    if r2 < 1e-9: r2 = 1e-9
    term1 = np.exp(-alpha * r1)
    term2 = np.exp(-alpha * r2)
    return term1 + sign * term2

In [47]:
JASTROW_A = 0.5

@njit(fastmath=True)
def calc_Jastrow(r12_mag, beta_val):
    """
    This function Calculates the value of the Jastrow correlation factor.

    Parameters:
    r12_mag (float): The distance between the two electrons.
    beta_val (float): The variational parameter beta.

    Returns:
    float: The value of the Jastrow factor.
    """
    if r12_mag < 1e-9: r12_mag = 1e-9
    return JASTROW_A * r12_mag / (1.0 + beta_val * r12_mag)

In [48]:
@njit(fastmath=True)
def total_wavefunction(r_e1, r_e2, R_val, alpha, beta_val, P1, P2, sign=1):
    """
    This function calculates the total wavefunction of the system.

    Parameters:
    r_e1 (np.ndarray): The position of the first electron in 3D space.
    r_e2 (np.ndarray): The position of the second electron in 3D space.
    R_val (float): The distance between the two protons.
    alpha (float): The variational parameter.
    beta_val (float): The variational parameter for Jastrow factor.
    P1 (np.ndarray): The position of the first proton in 3D space.
    P2 (np.ndarray): The position of the second proton in 3D space.
    sign (int): The sign of the wavefunction.

    Returns:
    float: The value of the total wavefunction at the given electron positions.
    """
    psi_1 = single_electron_wavefunction(r_e1, R_val, alpha, P1, P2, sign=sign)
    psi_2 = single_electron_wavefunction(r_e2, R_val, alpha, P1, P2, sign=sign)

    r12_mag = vec_mag(r_e1 - r_e2)
    if r12_mag < 1e-9:
        r12_mag = 1e-9
        Jastrow_factor = np.exp(calc_Jastrow(r12_mag, beta_val))
    else:
        Jastrow_factor = np.exp(calc_Jastrow(r12_mag, beta_val))

    return psi_1 * psi_2 * Jastrow_factor

In [49]:
@njit(fastmath=True)
def H2_local_energy(r_e1, r_e2, R_val, alpha, beta_val, P1, P2, sign=1):
    """
    This function calculates the local energy of the system.

    Parameters:
    r_e1 (np.ndarray): The position of the first electron in 3D space.
    r_e2 (np.ndarray): The position of the second electron in 3D space.
    R_val (float): The distance between the two protons.
    alpha (float): The variational parameter.
    beta_val (float): The variational parameter for Jastrow factor.
    P1 (np.ndarray): The position of the first proton in 3D space.
    P2 (np.ndarray): The position of the second proton in 3D space.
    sign (int): The sign of the wavefunction.

    Returns:
    float: The local energy of the system at the given electron positions.
    """
    r1p1_mag = vec_mag(r_e1 - P1)
    r1p2_mag = vec_mag(r_e1 - P2)
    r2p1_mag = vec_mag(r_e2 - P1)
    r2p2_mag = vec_mag(r_e2 - P2)
    r12_mag = vec_mag(r_e1 - r_e2)

    #avoid division by zero
    if r1p1_mag < 1e-9: r1p1_mag = 1e-9
    if r1p2_mag < 1e-9: r1p2_mag = 1e-9
    if r2p1_mag < 1e-9: r2p1_mag = 1e-9
    if r2p2_mag < 1e-9: r2p2_mag = 1e-9
    if r12_mag < 1e-9: r12_mag = 1e-9
    if R_val < 1e-9: R_val = 1e-9

    #potential energy terms
    V = -1.0/r1p1_mag - 1.0/r1p2_mag - 1.0/r2p1_mag - 1.0/r2p2_mag \
        + 1.0/r12_mag + 1.0/R_val

    #kinetic energy of electron 1
    step = 1e-4
    KE1 = 0.0
    r1 = r_e1
    r2 = r_e2
    for i in range(r1.shape[0]): # Iterate over x,y,z
        r1_plus = r1.copy()
        r1_plus[i] += step
        psi_plus = total_wavefunction(r1_plus, r2, R_val, alpha, beta_val, P1, P2, sign)

        r1_minus = r1.copy()
        r1_minus[i] -= step
        psi_minus = total_wavefunction(r1_minus, r2, R_val, alpha, beta_val, P1, P2, sign)

        KE1 += (psi_plus + psi_minus - 2.0 * total_wavefunction(r1, r2, R_val, alpha, beta_val, P1, P2, sign))

    KE1 = -0.5 * KE1 / (step**2 * total_wavefunction(r1, r2, R_val, alpha, beta_val, P1, P2, sign))
    #kinetic energy of electron 2
    KE2 = 0.0
    for i in range(r2.shape[0]):
        r2_plus = r2.copy()
        r2_plus[i] += step
        psi_plus = total_wavefunction(r1, r2_plus, R_val, alpha, beta_val, P1, P2, sign)

        r2_minus = r2.copy()
        r2_minus[i] -= step
        psi_minus = total_wavefunction(r1, r2_minus, R_val, alpha, beta_val, P1, P2, sign)

        KE2 += (psi_plus + psi_minus - 2.0 * total_wavefunction(r1, r2, R_val, alpha, beta_val, P1, P2, sign))

    KE2 = -0.5 * KE2 / (step**2 * total_wavefunction(r1, r2, R_val, alpha, beta_val, P1, P2, sign))

    #total energy of the electron-proton system
    E = V + KE1 + KE2

    return E

In [50]:
@njit 
def H2_VMC(r, R_val, Z, step_size, samples=10000, alpha_val=2.0, beta_val=1.0, sign=1):
    """
    This function performs Variational Monte Carlo (VMC) simulation for the H2 molecule.

    Parameters:
        r (np.ndarray):  A 2x3 array containing the initial 3D positions of the two electrons.
        R_val (float): The inter-proton distance (in Bohr radii).
        Z (float): The nuclear charge (1.0 for hydrogen).
        step_size (float): The step size for the Metropolis algorithm.
        samples (int): The number of VMC steps to perform (default: 10000).
        alpha_val (float): The variational parameter alpha (default: 2.0).
        beta_val (float): The variational parameter beta (default: 1.0).
        sign (int): +1 for the bonding orbital, -1 for the anti-bonding orbital (default: 1).
    Returns:
       tuple: A tuple containing:
            - position_saved (list): A list of 2x3 NumPy arrays, storing the electron positions at each VMC step.
            - energy_saved (list): A list of floats, storing the local energy at each VMC step.
            - acceptance_rate (float): The acceptance rate of the Metropolis algorithm.
    """
    position_saved = []
    energy_saved = []
    P1, P2 = proton_points(R_val)
    r1 = r[0].copy()
    r2 = r[1].copy()
    psi_T_current = total_wavefunction(r1, r2, R_val, alpha_val, beta_val, P1, P2, sign)
    accepted_moves = 0

    for n in range(samples):
        r1_new = r1 + step_size * (np.random.rand(3) - 0.5)
        r2_new = r2 + step_size * (np.random.rand(3) - 0.5)

        psi_T_new = total_wavefunction(r1_new, r2_new, R_val, alpha_val, beta_val, P1, P2, sign)

        if abs(psi_T_current) < 1e-15:
             if abs(psi_T_new) > 1e-15:
                 acceptance_ratio = 1.0
             else:
                 acceptance_ratio = 0.0
        else:
            acceptance_ratio = (psi_T_new / psi_T_current)**2

        if np.random.rand() < acceptance_ratio:
            r1 = r1_new
            r2 = r2_new
            psi_T_current = psi_T_new
            accepted_moves += 1

        position_saved.append(np.stack((r1,r2)).copy())
        energy_saved.append(H2_local_energy(r1, r2, R_val, alpha_val, beta_val, P1, P2, sign))

    acceptance_rate = accepted_moves / samples
    return position_saved, energy_saved, acceptance_rate

In [51]:
def alpha_opt(alpha_list, R_val, Z, step, samples=10000, sign=1):
    """
    This function optimizes the alpha variational parameter
    for the H2 molecule using VMC.

    Parameters:
        alpha_list (list): A list of alpha values to test.
        R_val (float): The inter-proton distance.
        Z (float): The nuclear charge.
        step (float): The VMC step size.
        samples (int): The number of VMC steps to perform for each alpha (default: 10000).
        sign (int): +1 for the bonding orbital, -1 for the anti-bonding orbital (default: 1).

    Returns:
        tuple: A tuple containing:
            - saved_energies (list): A list of the mean energies for each alpha value.
            - optimal_alpha (float): The alpha value that minimizes the energy.
            - variance (list): A list of the energy variances for each alpha value.
            - mean_energies (list): A list of the mean energies for each alpha value (same as saved_energies).
    """
    mean_energies = []
    for a in alpha_list:
        r = np.random.rand(2, 3)
        _, energies, _ = H2_VMC(r, R_val, Z, step, samples, a, 1, sign=sign)
        mean_e = np.mean(energies)
        mean_energies.append(mean_e)

    optimal_alpha = alpha_list[np.argmin(mean_energies)]
    return optimal_alpha, mean_energies

In [52]:
def beta_opt(beta_list, optimal_alpha, R_val, Z, step, samples=10000, sign=1):
    """
    This function optimizes the beta variational parameter
    for the H2 molecule using VMC.

    Parameters:
        beta_list (list): A list of beta values to test.
        optimal_alpha (float): The optimized alpha value to use.
        R_val (float): The inter-proton distance.
        Z (float): The nuclear charge.
        step (float): The VMC step size.
        samples (int): The number of VMC steps to perform for each beta (default: 10000).
        sign (int): +1 for the bonding orbital, -1 for the anti-bonding orbital (default: 1).

    Returns:
        tuple: A tuple containing:
            - saved_energies (list): A list of the mean energies for each beta value.
            - optimal_beta (float): The beta value that minimizes the energy.
            - variance (list): A list of the energy variances for each beta value.
            - mean_energies (list): A list of the mean energies for each beta value (same as saved_energies).
    """
    mean_energies = []
    for b in beta_list:
        r = np.random.rand(2, 3)
        _, energies, _ = H2_VMC(r, R_val, Z, step, samples, optimal_alpha, b, sign=sign)
        mean_e = np.mean(energies)
        mean_energies.append(mean_e)
    
    optimal_beta = beta_list[np.argmin(mean_energies)]
    return optimal_beta, mean_energies

In [53]:
R_val = 1.4
Z = 1.0
N_STEPS = 200000
INITIAL_STEP_SIZE = 0.5

alpha_list = np.linspace(0.5, 2.0, 10)
optimal_alpha, energies_alpha = alpha_opt(alpha_list, R_val, Z, INITIAL_STEP_SIZE, N_STEPS)
print(f"Optimal alpha: {optimal_alpha}")

beta_list = np.linspace(0.1, 2.0, 10)
optimal_beta, energies_beta = beta_opt(beta_list, optimal_alpha, R_val, Z, INITIAL_STEP_SIZE, N_STEPS)
energies_beta = np.where(np.isnan(energies_beta), np.inf, energies_beta)
optimal_beta = beta_list[np.argmin(energies_beta)]
print(f"Optimal beta: {optimal_beta}")

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Energy vs Alpha", "Energy vs Beta (Optimal Alpha)"))

fig.add_trace(go.Scatter(x=alpha_list, y=energies_alpha, mode='lines+markers', name='Energy vs Alpha'), row=1, col=1)
fig.update_xaxes(title_text="Alpha", row=1, col=1)
fig.update_yaxes(title_text="Energy", row=1, col=1)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='gray', row=1, col=1)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='gray', row=1, col=1)

fig.add_trace(go.Scatter(x=beta_list, y=energies_beta, mode='lines+markers', name='Energy vs Beta'), row=1, col=2)
fig.update_xaxes(title_text="Beta", row=1, col=2)
fig.update_yaxes(title_text="Energy", row=1, col=2)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='gray', row=1, col=2)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='gray', row=1, col=2)

fig.update_layout(
    title="Energy Optimization for Alpha and Beta",
    width=1200, height=500, showlegend=True,
    plot_bgcolor='black', paper_bgcolor='black', font=dict(color='white'),
    title_font=dict(size=18, color='white'),
    legend=dict(x=0.8, y=1, traceorder='normal', font=dict(size=12, color='white'), bgcolor='rgba(0, 0, 0, 0.5)'),
)

fig.show()

print(f"Optimal alpha: {optimal_alpha}")
print(f"Optimal beta: {optimal_beta}")

Optimal alpha: 1.1666666666666665
Optimal beta: 0.9444444444444444


Optimal alpha: 1.1666666666666665
Optimal beta: 0.9444444444444444


In [54]:
R_values = np.linspace(0.5, 3.0, 20)
N_STEPS = 200000
INITIAL_STEP_SIZE = 0.5
Z = 1.0
alpha_val = optimal_alpha
beta_val = optimal_beta

results_bonding = []

print("Starting VMC for bonding (+) wavefunction...")
for R_val in R_values:
    r = np.random.rand(2, 3)
    _, energies, _ = H2_VMC(r, R_val, Z, INITIAL_STEP_SIZE, N_STEPS, alpha_val, beta_val, sign=1)

    mean_e = np.mean(energies)
    results_bonding.append({'R': R_val, 'E': mean_e, 'alpha': alpha_val, 'beta': beta_val})

results_bonding_arr = np.array([(r['R'], r['E']) for r in results_bonding])

Starting VMC for bonding (+) wavefunction...


In [55]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results_bonding_arr[:, 0],
    y=results_bonding_arr[:, 1],
    mode='lines+markers',
    name='VMC Energy (Bonding)',
    line=dict(color='royalblue')
))

fig.add_trace(go.Scatter(
    x=[R_values[0], R_values[-1]],
    y=[-1.17460, -1.17460],
    mode='lines',
    name='Exact Ground State Energy (-1.1746 Ha)',
    line=dict(color='red', dash='dash')
))

fig.add_trace(go.Scatter(
    x=[1.40, 1.40],
    y=[min(results_bonding_arr[:, 1]), max(results_bonding_arr[:, 1])],
    mode='lines',
    name='Equilibrium Distance (1.40 Bohr)',
    line=dict(color='white', dash='dot')
))

min_energy_idx = np.argmin(results_bonding_arr[:, 1])
min_energy_R = results_bonding_arr[min_energy_idx, 0]
min_energy = results_bonding_arr[min_energy_idx, 1]

fig.add_trace(go.Scatter(
    x=[min_energy_R, min_energy_R],
    y=[min(results_bonding_arr[:, 1]), max(results_bonding_arr[:, 1])],
    mode='lines',
    name=f'Calculated Equilibrium Distance ({min_energy_R:.2f} Bohr)',
    line=dict(color='yellow', dash='dot')
))

fig.add_trace(go.Scatter(
    x=[R_values[0], R_values[-1]],
    y=[min_energy, min_energy],
    mode='lines',
    name=f'Minimum Energy ({min_energy:.5f} Ha)',
    line=dict(color='green', dash='dot')
))

fig.update_layout(
    title="H2 Ground State Energy (Bonding WF) vs. R",
    xaxis_title="Inter-proton distance R (Bohr)",
    yaxis_title="Energy (Hartree)",
    width=1200, height=600, showlegend=True,
    plot_bgcolor='black', paper_bgcolor='black', font=dict(color='white'),
    title_font=dict(size=18, color='white'),
    legend=dict(x=0.8, y=1, traceorder='normal', font=dict(size=12, color='white'), bgcolor='rgba(0, 0, 0, 0.5)'),
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray')
)

fig.show()

In [56]:
print("Starting VMC for anti-bonding (-) wavefunction...")
results_anti_bonding = []
for R_val in R_values:
    r = np.random.rand(2, 3)
    _, energies, _ = H2_VMC(r, R_val, Z, INITIAL_STEP_SIZE, N_STEPS, alpha_val, beta_val, sign=-1)

    mean_e = np.mean(energies)
    results_anti_bonding.append({'R': R_val, 'E': mean_e, 'alpha': alpha_val, 'beta': beta_val})

results_anti_bonding_arr = np.array([(r['R'], r['E']) for r in results_anti_bonding])

Starting VMC for anti-bonding (-) wavefunction...


In [57]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results_anti_bonding_arr[:, 0],
    y=results_anti_bonding_arr[:, 1],
    mode='lines+markers',
    name='VMC Energy (Anti-Bonding)',
    line=dict(color='royalblue')
))

fig.add_trace(go.Scatter(
    x=[R_values[0], R_values[-1]],
    y=[-1.17460, -1.17460],
    mode='lines',
    name='Exact Ground State Energy (-1.1746 Ha)',
    line=dict(color='red', dash='dash')
))

fig.add_trace(go.Scatter(
    x=[1.40, 1.40],
    y=[min(results_anti_bonding_arr[:, 1]), max(results_anti_bonding_arr[:, 1])],
    mode='lines',
    name='Equilibrium Distance (1.40 Bohr)',
    line=dict(color='gray', dash='dot')
))

min_energy_idx = np.argmin(results_anti_bonding_arr[:, 1])
min_energy_R = results_anti_bonding_arr[min_energy_idx, 0]
min_energy = results_anti_bonding_arr[min_energy_idx, 1]

fig.add_trace(go.Scatter(
    x=[min_energy_R, min_energy_R],
    y=[min(results_anti_bonding_arr[:, 1]), max(results_anti_bonding_arr[:, 1])],
    mode='lines',
    name=f'Calculated Equilibrium Distance ({min_energy_R:.2f} Bohr)',
    line=dict(color='lime', dash='dot')
))

fig.add_trace(go.Scatter(
    x=[R_values[0], R_values[-1]],
    y=[min_energy, min_energy],
    mode='lines',
    name=f'Minimum Energy ({min_energy:.5f} Ha)',
    line=dict(color='lime', dash='dot')
))

fig.update_layout(
    title="H2 Ground State Energy (Anti-Bonding WF) vs. R",
    xaxis_title="Inter-proton distance R (Bohr)",
    yaxis_title="Energy (Hartree)",
    width=1200, height=600, showlegend=True,
    plot_bgcolor='black', paper_bgcolor='black', font=dict(color='white'),
    title_font=dict(size=18, color='white'),
    legend=dict(x=0.8, y=1, traceorder='normal', font=dict(size=12, color='white'), bgcolor='rgba(0, 0, 0, 0.5)'),
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray')
)

fig.show()

In [58]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results_bonding_arr[:, 0],
    y=results_bonding_arr[:, 1],
    mode='lines+markers',
    name='VMC Energy (Bonding)',
    line=dict(color='royalblue')
))

fig.add_trace(go.Scatter(
    x=results_anti_bonding_arr[:, 0],
    y=results_anti_bonding_arr[:, 1],
    mode='lines+markers',
    name='VMC Energy (Anti-Bonding)',
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=[R_values[0], R_values[-1]],
    y=[-1.17460, -1.17460],
    mode='lines',
    name='Exact Ground State Energy (-1.1746 Ha)',
    line=dict(color='red', dash='dash')
))

min_bonding_idx = np.argmin(results_bonding_arr[:, 1])
min_bonding_R = results_bonding_arr[min_bonding_idx, 0]
min_bonding_energy = results_bonding_arr[min_bonding_idx, 1]

fig.add_trace(go.Scatter(
    x=[min_bonding_R, min_bonding_R],
    y=[min(results_bonding_arr[:, 1]), max(results_bonding_arr[:, 1])],
    mode='lines',
    name=f'Equilibrium Distance (Bonding) ({min_bonding_R:.2f} Bohr)',
    line=dict(color='lime', dash='dot')
))

fig.add_trace(go.Scatter(
    x=[R_values[0], R_values[-1]],
    y=[min_bonding_energy, min_bonding_energy],
    mode='lines',
    name=f'Minimum Energy (Bonding) ({min_bonding_energy:.5f} Ha)',
    line=dict(color='lime', dash='dot')
))

min_anti_bonding_idx = np.argmin(results_anti_bonding_arr[:, 1])
min_anti_bonding_R = results_anti_bonding_arr[min_anti_bonding_idx, 0]
min_anti_bonding_energy = results_anti_bonding_arr[min_anti_bonding_idx, 1]

fig.add_trace(go.Scatter(
    x=[min_anti_bonding_R, min_anti_bonding_R],
    y=[min(results_anti_bonding_arr[:, 1]), max(results_anti_bonding_arr[:, 1])],
    mode='lines',
    name=f'Equilibrium Distance (Anti-Bonding) ({min_anti_bonding_R:.2f} Bohr)',
    line=dict(color='cyan', dash='dot')
))

fig.add_trace(go.Scatter(
    x=[R_values[0], R_values[-1]],
    y=[min_anti_bonding_energy, min_anti_bonding_energy],
    mode='lines',
    name=f'Minimum Energy (Anti-Bonding) ({min_anti_bonding_energy:.5f} Ha)',
    line=dict(color='cyan', dash='dot')
))

fig.update_layout(
    title="H2 Ground State Energy (Bonding and Anti-Bonding WF) vs. R",
    xaxis_title="Inter-proton distance R (Bohr)",
    yaxis_title="Energy (Hartree)",
    width=1200, height=600, showlegend=True,
    plot_bgcolor='black', paper_bgcolor='black', font=dict(color='white'),
    title_font=dict(size=18, color='white'),
    legend=dict(x=0.75, y=1, traceorder='normal', font=dict(size=12, color='white'), bgcolor='rgba(0, 0, 0, 0.5)'),
    xaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray'),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='gray')
)

fig.show()